# Introduction
This notebook does feature engineering given the raw details of a trip i.e. second-by-second speed, accelerometer, gyro, etc readings.

# Libraries

In [3]:
import pyspark.sql.functions as f
import itertools
from pyspark.sql.window import Window
import re

# Data

Load the raw features of the valid trips:

*Note: To compute the features of the hold-out set, just save the hold-out set as parquet and modify the path accordingly*

In [6]:
feature_df = spark.read.parquet('dbfs:/msh/grab/data/features_valid.parquet')

# Feature Engineering

In [8]:
feature_df.columns

Out[158]: 
['bookingID',
 'Accuracy',
 'Bearing',
 'acceleration_x',
 'acceleration_y',
 'acceleration_z',
 'gyro_x',
 'gyro_y',
 'gyro_z',
 'second',
 'Speed']

Compute magnitude of the accelerometer and gyrosope vectors:

In [10]:
mag_acc = f.sqrt(f.pow(f.col('acceleration_x'), 2) + 
                 f.pow(f.col('acceleration_y'), 2) + 
                 f.pow(f.col('acceleration_z'), 2))

mag_gyro = f.sqrt(f.pow(f.col('gyro_x'), 2) + 
                  f.pow(f.col('gyro_y'), 2) + 
                  f.pow(f.col('gyro_y'), 2))

feature_df = feature_df\
  .withColumn('mag_acc', mag_acc)\
  .withColumn('mag_gyro', mag_gyro)

Summarize the features of all trips:

In [12]:
def summarizer(col_name):
  return [f.mean(col_name).alias(f'mean_{col_name}'),
          f.stddev(col_name).alias(f'sd_{col_name}'),
          f.min(col_name).alias(f'min_{col_name}'),
          f.max(col_name).alias(f'max_{col_name}')]

agg_features = [summarizer(col) for col in ['Speed', 'mag_acc', 'mag_gyro']]
agg_features = itertools.chain(*agg_features)
agg_features = list(agg_features)

summary_df = feature_df\
  .groupBy('bookingID')\
  .agg(f.max('second').alias('trip_length'),
       *agg_features)

display(summary_df.head(10))

bookingID,trip_length,mean_Speed,sd_Speed,min_Speed,max_Speed,mean_mag_acc,sd_mag_acc,min_mag_acc,max_mag_acc,mean_mag_gyro,sd_mag_gyro,min_mag_gyro,max_mag_gyro
180388626497,1227.0,14.584302757022652,7.3805421529237405,0.26791424,28.887203,9.887102131125037,0.9639352437941556,6.282746070488001,13.752636222198753,0.19072322715941148,0.14876828870258998,0.0016417552874458644,0.9782266904117365
420906795087,1425.0,9.999966998436996,7.817337211485149,9.901444173010532E-6,25.990108489990234,9.853606368378275,0.6780796877920069,6.560842090406892,15.126575687452954,0.10887106208931505,0.1155621569803626,7.182461032204933E-4,0.7725758014877556
738734375015,1595.0,15.521507640710146,7.354990390833642,0.098512895,24.09367,9.898056305963397,0.9413202140725359,6.084748843549244,16.008818538294104,0.19679062394268013,0.15460160152424354,0.0019264527169510313,1.3310844860831132
1116691497112,2442.0,5.39321308269489,4.693224330866682,0.005812864750623703,19.50569725036621,9.90236587306102,0.9927144491271708,0.5215951069234029,18.807262036581307,0.1156831755456118,0.13168976264583468,8.989191274704996E-4,1.0714647613839892
1520418422860,1552.0,17.32062543921293,7.948089419305171,0.15,27.39,9.702427637418003,0.49234596742643155,6.644502145838063,12.574411719783509,0.05047204628410813,0.07912601347794802,4.3827535095596824E-4,0.7330567342520905
1142461300782,1411.0,11.005938713777796,8.681401720449887,0.3100000023841858,27.93000030517578,9.825148695211356,0.7241919999151112,5.245038163310359,14.905587527189512,0.14415704098886978,0.13664480943935894,7.419004223478598E-4,1.0826688006530418
412316860579,1319.0,17.842677491105263,9.526602732834446,0.01,28.97,9.963744177969993,0.8185820745119907,6.206545941009601,14.725856772652755,0.14795359159961013,0.10390516598922535,0.00183259575,0.8423463525059106
1597727834164,1229.0,12.735908919043345,8.950204491723966,0.011295424774289131,29.733797073364258,9.857803975127874,0.9745416809962099,4.046950334857807,19.633861721884614,0.22652979978231974,0.18716308302413237,0.002836963810833806,2.3343256544498447
1408749273107,1559.0,15.055722891566267,5.6052156991439,0.15,23.75,9.829436224660924,0.921204805691201,5.987091827370406,14.810688581307444,0.10514993150108111,0.08759880681590597,0.0020418170290880332,0.6067730033606404
120259084395,1470.0,8.958158441838684,5.951062644329034,0.31718874,25.781813,9.939689743627126,0.598469485303307,6.542895700720147,12.851942277998074,0.09650227931528373,0.1140801964716925,0.0018948472067864001,0.8546114013509102


Compute the percentiles of each feature:

In [14]:
source_columns = ['Speed', 'mag_acc', 'mag_gyro']
n_cols = len(source_columns)

window = Window.partitionBy('bookingID')

percentile_fns = [f.expr(f'percentile_approx({col}, array(0.05, 0.25, 0.5, 0.75, 0.95))')\
                   .over(window)\
                   .alias(f'{col}_quantiles')
                  for col in source_columns]

quantile_df = feature_df\
  .select('bookingID', *percentile_fns)

quantile_df = quantile_df\
  .groupBy('bookingID')\
  .agg({col: 'first' for col in quantile_df.columns[1:]})

# let each element in the list of quantiles be a separate column
flatten_quantiles = [f.col(col)[i] 
   for i in range(5) 
   for col in quantile_df.columns[1:]]

quantile_df = quantile_df\
  .select('bookingID', *flatten_quantiles)

# rename the quantile columns to something more readable
pat = r'first\(([a-zA-Z_]+)\)\[(\d)\]'

new_col_names = [re.sub(pat, r'\1_\2', col) 
                 for col in quantile_df.columns]
old_col_names = quantile_df.columns
select_col_expr = [f.col(old_name).alias(new_name) for old_name, new_name in zip(old_col_names, new_col_names)]

quantile_df = quantile_df.select(*select_col_expr)

display(quantile_df)

bookingID,Speed_quantiles_0,mag_acc_quantiles_0,mag_gyro_quantiles_0,Speed_quantiles_1,mag_acc_quantiles_1,mag_gyro_quantiles_1,Speed_quantiles_2,mag_acc_quantiles_2,mag_gyro_quantiles_2,Speed_quantiles_3,mag_acc_quantiles_3,mag_gyro_quantiles_3,Speed_quantiles_4,mag_acc_quantiles_4,mag_gyro_quantiles_4
1116691497112,0.1841275990009308,8.561550703912513,0.009816137322656638,1.4644402265548706,9.664903123423699,0.027062306664733873,4.115224361419678,9.844484323483856,0.06523249229807031,8.319931030273438,10.147073966968383,0.15647002787937497,15.084827423095703,11.39377022714889,0.4020868883576156
1142461300782,0.699999988079071,8.753983399291332,0.018078675183895974,4.130000114440918,9.520522868937578,0.0450208547710653,8.229999542236328,9.776796761319684,0.10403709522482141,16.149999618530273,10.126721510040547,0.19838047111972765,26.790000915527344,10.90192380438783,0.4049277497864431
120259084395,1.6205405,8.970611645539678,0.010362439688151242,3.9769514,9.621639830688878,0.030673184350150588,7.503737,9.913211717923696,0.05728601045408599,13.717563,10.2412225851562,0.10822891329264697,20.054876,10.880462002278412,0.3738823723992242
1408749273107,2.22,8.386101611589867,0.013317885068004565,12.1,9.261752750259351,0.04340771243104268,16.45,9.793076187520166,0.0814042440977534,19.21,10.383389139639238,0.13944306614590876,21.5,11.349736326439002,0.2791679919239239
1417339207742,1.28,8.917727257984753,0.007123851876864448,4.96,9.5572208209078,0.02055268734034985,10.82,9.880464092490996,0.0390476226458014,19.99,10.226317766726389,0.07322233459881716,23.73,10.927947092833607,0.337278943079134
146028888109,1.2566286,8.761400470387892,0.01310597470587125,4.944386,9.600077596821523,0.043875553357555414,11.136152,9.924904752792857,0.08310886904325442,16.734615,10.376281624048547,0.1480384799653107,22.098806,11.291370363548117,0.30280832927625256
1520418422860,2.88,8.978629374340143,0.005543499358518207,9.65,9.448040970491027,0.01486247540703304,21.42,9.679670689980558,0.025872296701992616,23.58,9.948662143157824,0.04440411421046643,25.41,10.477405742931893,0.21849740708141455
1546188226642,0.07499022036790848,8.742748533173101,0.018334175413107306,1.030515432357788,9.484986423071092,0.053540066571572745,8.73748779296875,9.841909519843268,0.12704230431028704,23.115997314453125,10.159462809208877,0.262459540951742,31.629785537719727,11.038960337605245,0.5809891641196243
1597727834164,0.061723072081804276,8.499897694930208,0.03646286295116499,4.243778705596924,9.388826887179011,0.10645929189747669,13.513873100280762,9.806304252972964,0.1862086762305907,20.68726348876953,10.27508378021588,0.2829113265278138,26.723926544189453,11.432263004149243,0.5632386979296101
180388626497,2.8184507,8.32953049464233,0.026770408677136048,7.7261295,9.360774901029682,0.0898408356664491,15.313547,9.835571841359496,0.15573423574369186,21.141994,10.391816736920747,0.24895823976112358,24.578161,11.621482503968002,0.4820164261093142


Join `quantile_df` to `summary_df`:

In [16]:
summary_df = summary_df.join(quantile_df, ['bookingID'], 'inner')

display(summary_df)

bookingID,trip_length,mean_Speed,sd_Speed,min_Speed,max_Speed,mean_mag_acc,sd_mag_acc,min_mag_acc,max_mag_acc,mean_mag_gyro,sd_mag_gyro,min_mag_gyro,max_mag_gyro,Speed_quantiles_0,mag_acc_quantiles_0,mag_gyro_quantiles_0,Speed_quantiles_1,mag_acc_quantiles_1,mag_gyro_quantiles_1,Speed_quantiles_2,mag_acc_quantiles_2,mag_gyro_quantiles_2,Speed_quantiles_3,mag_acc_quantiles_3,mag_gyro_quantiles_3,Speed_quantiles_4,mag_acc_quantiles_4,mag_gyro_quantiles_4
1116691497112,2442.0,5.39321308269489,4.693224330866682,0.005812864750623703,19.50569725036621,9.90236587306102,0.9927144491271707,0.5215951069234029,18.807262036581307,0.1156831755456118,0.13168976264583468,8.989191274704996E-4,1.0714647613839892,0.1841275990009308,8.561550703912513,0.009816137322656638,1.4644402265548706,9.664903123423699,0.027062306664733873,4.115224361419678,9.844484323483856,0.06523249229807031,8.319931030273438,10.147073966968383,0.15647002787937497,15.084827423095703,11.39377022714889,0.4020868883576156
1142461300782,1411.0,11.005938713777796,8.681401720449887,0.3100000023841858,27.93000030517578,9.825148695211356,0.7241919999151111,5.245038163310359,14.905587527189512,0.14415704098886978,0.13664480943935894,7.419004223478598E-4,1.0826688006530418,0.699999988079071,8.753983399291332,0.018078675183895974,4.130000114440918,9.520522868937578,0.0450208547710653,8.229999542236328,9.776796761319684,0.10403709522482141,16.149999618530273,10.126721510040547,0.19838047111972765,26.790000915527344,10.90192380438783,0.4049277497864431
120259084395,1470.0,8.958158441838684,5.951062644329034,0.31718874,25.781813,9.939689743627126,0.5984694853033071,6.542895700720147,12.851942277998074,0.09650227931528373,0.1140801964716925,0.0018948472067864001,0.8546114013509102,1.6205405,8.970611645539678,0.010362439688151242,3.9769514,9.621639830688878,0.030673184350150588,7.503737,9.913211717923696,0.05728601045408599,13.717563,10.2412225851562,0.10822891329264697,20.054876,10.880462002278412,0.3738823723992242
1408749273107,1559.0,15.055722891566267,5.6052156991438995,0.15,23.75,9.829436224660924,0.9212048056912009,5.987091827370406,14.810688581307444,0.10514993150108114,0.08759880681590597,0.0020418170290880332,0.6067730033606404,2.22,8.386101611589867,0.013317885068004565,12.1,9.261752750259351,0.04340771243104268,16.45,9.793076187520166,0.0814042440977534,19.21,10.383389139639238,0.13944306614590876,21.5,11.349736326439002,0.2791679919239239
1417339207742,1274.0,12.093907113066228,7.850675121788485,0.01,25.359999,9.894925076946075,0.6332253200215338,7.035821076908314,13.261218558315901,0.07589190961923847,0.12020602877945492,0.0018325958,1.2293817709788277,1.28,8.917727257984753,0.007123851876864448,4.96,9.5572208209078,0.02055268734034985,10.82,9.880464092490996,0.0390476226458014,19.99,10.226317766726389,0.07322233459881716,23.73,10.927947092833607,0.337278943079134
146028888109,1214.0,11.101259584068071,6.903326629754551,0.30856746,26.894707,9.971840280209204,0.7560823463813107,7.309832129883759,13.147448193317908,0.11452726571141296,0.10864435566023797,9.286932068436324E-4,0.7685288672604699,1.2566286,8.761400470387892,0.01310597470587125,4.944386,9.600077596821523,0.043875553357555414,11.136152,9.924904752792857,0.08310886904325442,16.734615,10.376281624048547,0.1480384799653107,22.098806,11.291370363548117,0.30280832927625256
1520418422860,1552.0,17.32062543921293,7.948089419305171,0.15,27.39,9.702427637418001,0.4923459674264315,6.644502145838063,12.574411719783509,0.05047204628410813,0.07912601347794801,4.3827535095596824E-4,0.7330567342520905,2.88,8.978629374340143,0.005543499358518207,9.65,9.448040970491027,0.01486247540703304,21.42,9.679670689980558,0.025872296701992616,23.58,9.948662143157824,0.04440411421046643,25.41,10.477405742931893,0.21849740708141455
1546188226642,1364.0,12.567189085124667,11.595579338743619,0.02702218107879162,33.58143615722656,9.85191728474334,0.7538792777286968,4.204473238503064,16.922898622503848,0.19479834186666162,0.21670147964540779,0.0

# Save

Save the feature summary of each trip:

In [19]:
summary_df.write.parquet('dbfs:/msh/grab/data/features_valid_summary.parquet')